In [1]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from pyspark.sql import *
#from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import math

In [12]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [13]:
sqlContext = SQLContext(sc)
geo_tags = sqlContext.read.csv('geo_tags.csv', header=True, inferSchema=True)
geo_tags_rdd = geo_tags.rdd

In [4]:
contents = sqlContext.read.csv('contents.csv', header=True, inferSchema=True)
contents_rdd = contents.rdd

In [5]:
geo_tags_latitud_y_longitud=geo_tags_rdd.map(lambda x: (x[1], [x[4], x[5]])).filter(lambda x: x[0]!= None )

In [6]:
contents_mapped=contents_rdd.filter(lambda x: x[1]!= None and x[1].isnumeric()).map(lambda x: (int(x[1]), x[6] if x[6] != None else x[8]))

In [7]:
test_v=geo_tags_latitud_y_longitud.join(contents_mapped).map(lambda x: (x[1][1], x[1][0])).reduceByKey(lambda x,y: [(x[0]+y[0])/2, (x[1]+y[1])/2]).sortBy(lambda x: x[1]).cache()

In [8]:
def dist(l1, l2):
    return math.sqrt((l1[0]-l2[0])**2+(l1[1]-l2[1])**2)

In [9]:
lista = []
listas=test_v.collect()
for i in range(len(listas)-1):
    lista.append((listas[i], listas[i+1]))

In [10]:
rdd_grafo = sc.parallelize(lista)

In [11]:
rdd_grafo.map(lambda x: (x[0][0], x[1][0], dist(x[0][1], x[1][1]))).sortBy(lambda x: x[2]).take(1)

[('190.104.131.138', '45.227.216.227', 0.0)]